по терминологии: чтобы не возникло путаницы в понятии вершины, я буду называть вершину графа вертексом, а вершину в синтаксическом дереве вершиной
а ребро графа либо ребром, либо эджом. чанк - это именная группа

In [2]:
import spacy
import spacy.cli

spacy.cli.download("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 334.8 kB/s eta 0:00:0000:0101:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp_en = spacy.load("en_core_web_lg")
doc = nlp_en("Colorless green ideas sleep furiously with blue elephants of monkeys")

In [4]:
from spacy import displacy

In [84]:
displacy.render(doc, style='dep')

In [77]:
def get_connection(node: spacy.tokens.token.Token, stack: list) -> tuple[list, spacy.tokens.token.Token]:
    '''Recursive function which gets node and returns edge and other noun phrase which connected to it.'''
    print(node, node.pos_, stack)
    stack.append(node)
    if not node or node.pos_ == 'NOUN':
        print('yes!')
        return (node, stack)
    if node.children:
        for child in node.children:
            stack = []
            get_connection(child, stack)
    else:
        print('yes!')
        return (node, stack)

- бежим по чанкам (чанк - вертекс)
- у каждого чанка ищем вершину (она будет эджом)
- проверяем, есть ли уже в графе этот эджс с такой же дочерней вершиной. если есть, то значит это тот же эдж и мы просто добавляем ему второй вертекс. если нет, то создаём новый эдж

In [81]:
# from graph import Graph
# from vertex import Vertex
# graph = Graph()
for chunk in doc.noun_chunks:
    # graph.add_vertex(chunk.text, chunk.text.split())
    print(chunk)
    print(get_connection(chunk.root.head, []), '!!!')
    # print (lst, vert)
    # print(type(chunk), chunk.root.pos_)

Colorless green ideas
sleep VERB []
ideas NOUN []
yes!
furiously ADV []
with ADP []
elephants NOUN []
yes!
None !!!
blue elephants
with ADP []
elephants NOUN []
yes!
None !!!
monkeys
elephants NOUN []
yes!
(elephants, [elephants]) !!!


In [23]:
print(repr(graph))

Graph(
	vertices=[Vertex(concept='Girafes', words=['Girafes']), Vertex(concept='colorless green ideas', words=['colorless', 'green', 'ideas']), Vertex(concept='blue elephants', words=['blue', 'elephants'])],
	edges=[]
)


In [60]:
from nltk.parse.corenlp import CoreNLPDependencyParser

# Specify the path to the Stanford Parser JAR file and model file
stanford_parser_jar = '/path/to/stanford-parser.jar'
stanford_model_jar = '/path/to/englishPCFG.ser.gz'

# Create a StanfordDependencyParser object
dep_parser = CoreNLPDependencyParser(stanford_parser_jar, stanford_model_jar)

# Parse a sentence
sentence = "The quick brown fox jumps over the lazy dog."
result = list(dep_parser.raw_parse(sentence))

# Access dependency parsing information
for parse_tree in result:
    print(parse_tree.to_conll(4))

LookupError: unknown encoding: /path/to/englishPCFG.ser.gz